In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import plotly.graph_objects as go
import plotly.express as px

import matplotlib.pyplot as plt

sns.set_style(
    style='darkgrid', 
    rc={'axes.facecolor': '.9', 'grid.color': '.8'}
)
cmaps_hex = ['#193251','#FF5A36','#1E4485', '#99D04A','#FF5A36', '#DB6668']
sns.set_palette(palette=cmaps_hex)
sns_c = sns.color_palette(palette=cmaps_hex)

from scipy.ndimage import gaussian_filter

from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.formula.api as smf
from statsmodels.tsa.stattools import adfuller,kpss







plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 100

# EDA

## Data Loading

In [6]:
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')
df_store = pd.read_csv('data/store.csv')

In [7]:
df_train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [8]:
# function to rename columns in lower case
def lower_case(dataframe):
    cols = dataframe.columns.tolist()
    cols = [col.lower() for col in cols]
    dataframe.columns = cols
    return dataframe

In [25]:
lower_case(df_train);
lower_case(df_store);

In [10]:
# function to change date into datetime
def to_datetime(dataframe):
    dataframe.assign(
        timestamp = lambda x: pd.to_datetime(x['date']),
        year = lambda x: x['timestamp'].dt.year,
        month = lambda x: x['timestamp'].dt.month,
        day = lambda x: x['timestamp'].dt.day,
        dayofyear = lambda x: x['timestamp'].dt.dayofyear)
    return dataframe
# does not work :( 

In [11]:
#df_train = to_datetime(df_train)

In [13]:
df_train = df_train.assign(
            timestamp = lambda x: pd.to_datetime(x['date']),
            year = lambda x: x['timestamp'].dt.year,
            month = lambda x: x['timestamp'].dt.month,
            day = lambda x: x['timestamp'].dt.day,
            dayofyear = lambda x: x['timestamp'].dt.dayofyear)
df_train.drop("date", inplace=True, axis=1)

In [14]:
df_train.head()

,store,dayofweek,sales,customers,open,promo,stateholiday,schoolholiday,timestamp,year,month,day,dayofyear
0,1,5,5263,555,1,1,0,1,2015-07-31,2015,7,31,212
1,2,5,6064,625,1,1,0,1,2015-07-31,2015,7,31,212
2,3,5,8314,821,1,1,0,1,2015-07-31,2015,7,31,212
3,4,5,13995,1498,1,1,0,1,2015-07-31,2015,7,31,212
4,5,5,4822,559,1,1,0,1,2015-07-31,2015,7,31,212


In [26]:
print(df_store.shape)
df_store.head()

(1115, 10)


,store,storetype,assortment,competitiondistance,competitionopensincemonth,competitionopensinceyear,promo2,promo2sinceweek,promo2sinceyear,promointerval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


## Data Cleaning

In [15]:
df_train.shape

(1017209, 13)

In [19]:
df_train['stateholiday'].unique()

array(['0', 'a', 'b', 'c', 0], dtype=object)

In [20]:
df_train.stateholiday.value_counts()

0    855087
0    131072
a     20260
b      6690
c      4100
Name: stateholiday, dtype: int64

In [21]:
df_train['stateholiday'].replace({0:'0'}, inplace=True)
round(df_train.describe().T,2)

,count,mean,std,min,25%,50%,75%,max
store,1017209.0,558.43,321.91,1.0,280.0,558.0,838.0,1115.0
dayofweek,1017209.0,4.00,2.00,1.0,2.0,4.0,6.0,7.0
sales,1017209.0,5773.82,3849.93,0.0,3727.0,5744.0,7856.0,41551.0
customers,1017209.0,633.15,464.41,0.0,405.0,609.0,837.0,7388.0
open,1017209.0,0.83,0.38,0.0,1.0,1.0,1.0,1.0
promo,1017209.0,0.38,0.49,0.0,0.0,0.0,1.0,1.0
schoolholiday,1017209.0,0.18,0.38,0.0,0.0,0.0,0.0,1.0
year,1017209.0,2013.83,0.78,2013.0,2013.0,2014.0,2014.0,2015.0
month,1017209.0,5.85,3.33,1.0,3.0,6.0,8.0,12.0
day,1017209.0,15.70,8.79,1.0,8.0,16.0,23.0,31.0


### Handling NaN's

In [22]:
df_store.isnull().sum()

store                          0
storetype                      0
assortment                     0
competitiondistance            3
competitionopensincemonth    354
competitionopensinceyear     354
promo2                         0
promo2sinceweek              544
promo2sinceyear              544
promointerval                544
dtype: int64

In [27]:
df_store[pd.isnull(df_store.competitiondistance)]

,store,storetype,assortment,competitiondistance,competitionopensincemonth,competitionopensinceyear,promo2,promo2sinceweek,promo2sinceyear,promointerval
290,291,d,a,NaN,NaN,NaN,0,NaN,NaN,NaN
621,622,a,c,NaN,NaN,NaN,0,NaN,NaN,NaN
878,879,d,a,NaN,NaN,NaN,1,5.0,2013.0,"Feb,May,Aug,Nov"


In [30]:
# fill NaN with a median value
df_store['competitiondistance'].fillna(df_store['competitiondistance'].median(), inplace = True)
df_store['competitiondistance'].isnull().sum()

0

In [31]:
tmp = df_store[pd.isnull(df_store.competitionopensinceyear)]
tmp[tmp.competitiondistance != 0].shape

(354, 10)

Here these stores have a competition in their vicinity (CompetitionDistance =/= 0), but there is no information about the year this competition has been open. This value needs to be imputed in a meaningful way. Or just filled with '0'.

In [34]:
tmp = df_store[pd.isnull(df_store.promo2sinceweek)]
tmp[tmp.promo2 != 0].shape

(0, 10)

There are no stores with information about 'promo2sinceweek' which have 'NaN' in promo2.

In [35]:
df_store.isnull().sum()

store                          0
storetype                      0
assortment                     0
competitiondistance            0
competitionopensincemonth    354
competitionopensinceyear     354
promo2                         0
promo2sinceweek              544
promo2sinceyear              544
promointerval                544
dtype: int64

In [36]:
# replace NA's by 0
df_store.fillna(0, inplace = True)

In [37]:
df_train.isnull().sum()

store            0
dayofweek        0
sales            0
customers        0
open             0
promo            0
stateholiday     0
schoolholiday    0
timestamp        0
year             0
month            0
day              0
dayofyear        0
dtype: int64

In [ ]:
# Merge df_store and df_train
df = df_train.merge(df_store, how='left', left_on=df_train.Store, right_on=df_store.Store)
df.drop(['key_0', 'Store_y'], axis=1, inplace=True)
df = df.rename(columns={'Store_x':'Store'})